Run if new environment

In [ ]:
!pip install rasterio scikit-learn matplotlib numpy joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 68.0 MB/s eta 0:00:00


Upload files

In [ ]:
from google.colab import files
uploaded = files.upload()

ZIP_TRAIN = "/content/training_patches_64tile.zip"
ZIP_COMP  = "/content/composites.zip"
ZIP_RGB   = "/content/RGB Composites.zip"


Saving training_patches_64tile.zip to training_patches_64tile.zip
Saving composites.zip to composites.zip
Saving RGB Composites.zip to RGB Composites.zip


Unzip files

In [ ]:
import os, zipfile, re, json, warnings
from pathlib import Path

import numpy as np
import rasterio
from rasterio.windows import Window
from rasterio.errors import NotGeoreferencedWarning
from rasterio.transform import Affine
from rasterio.warp import transform_bounds
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, cohen_kappa_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

import joblib, folium, imageio.v2 as iio
warnings.filterwarnings("ignore", category=NotGeoreferencedWarning)

OUT_DIR   = Path("/content/seagrass_rf_output")
TRAIN_DIR = OUT_DIR / "train_tiles"
AOI_DIR   = OUT_DIR / "aoi"
RGB_DIR   = OUT_DIR / "rgb"
PRED_DIR  = OUT_DIR / "predictions"
FIG_DIR   = OUT_DIR / "figures"
MAP_DIR   = OUT_DIR / "maps"
for d in [OUT_DIR, TRAIN_DIR, AOI_DIR, RGB_DIR, PRED_DIR, FIG_DIR, MAP_DIR]:
    d.mkdir(parents=True, exist_ok=True)

def safe_unzip(zip_path: str|Path, dest: Path):
    zip_path = Path(zip_path)
    if not zip_path.exists():
        print(f"[WARN] {zip_path} not found; skipping.")
        return
    with zipfile.ZipFile(zip_path, "r") as zf:
        for m in zf.namelist():
            p = Path(m)
            if p.name.startswith("__MACOSX") or p.name.startswith(".DS_Store"):
                continue
            target = dest / p
            if m.endswith("/"):
                target.mkdir(parents=True, exist_ok=True)
            else:
                target.parent.mkdir(parents=True, exist_ok=True)
                with zf.open(m) as src, open(target, "wb") as dst:
                    dst.write(src.read())
    print(f"[OK] Extracted {zip_path.name} → {dest}")

# unzip all three
safe_unzip(ZIP_TRAIN, TRAIN_DIR)
safe_unzip(ZIP_COMP, AOI_DIR)
safe_unzip(ZIP_RGB,  RGB_DIR)

def find_rasters(root: Path, exts=(".tif", ".tiff")) -> list[Path]:
    return sorted([p for p in root.rglob("*") if p.suffix.lower() in exts])

[OK] Extracted training_patches_64tile.zip → /content/seagrass_rf_output/train_tiles
[OK] Extracted composites.zip → /content/seagrass_rf_output/aoi
[OK] Extracted RGB Composites.zip → /content/seagrass_rf_output/rgb


Build training matrix

In [ ]:
# currently uses RGB + NIR
KEEP_BANDS = 4
TEST_SIZE = 0.2
RANDOM_STATE = 42
# number of trees
N_ESTIMATORS = 100
# None = unlimited
MAX_DEPTH = 20

def pair_images_labels(train_root: Path):
    img_dirs = [p for p in train_root.rglob("*") if p.is_dir() and p.name.lower() == "images"]
    lab_dirs = [p for p in train_root.rglob("*") if p.is_dir() and p.name.lower() == "labels"]
    pairs = []
    for img_dir in img_dirs:
        # find a sibling labels dir in parent or up the tree
        lab_dir = None
        if img_dir.parent / "labels" in lab_dirs:
            lab_dir = img_dir.parent / "labels"
        else:
            # fallback: any "labels" dir under train_root
            lab_dir = lab_dirs[0] if lab_dirs else None
        if lab_dir is None:
            continue
        lab_map = {p.stem: p for p in lab_dir.glob("*.tif")}
        for im in img_dir.glob("*.tif"):
            lb = lab_map.get(im.stem)
            if lb is not None:
                pairs.append((im, lb))
    return sorted(set(pairs))

def build_training_matrix_with_groups(pairs, keep_bands=KEEP_BANDS, valid=(0,1,2)):
    Xs, ys, gs = [], [], []
    for gid, (im, lb) in enumerate(pairs):
        with rasterio.open(im) as dsi, rasterio.open(lb) as dsl:
            B = min(keep_bands, dsi.count)
            img = dsi.read(list(range(1, B+1)))
            lab = dsl.read(1)
            X = img.reshape(B,-1).T
            y = lab.reshape(-1)

            # keep only finite labels (and valid set if provided)
            m = np.isfinite(y)
            if valid is not None:
                m &= np.isin(y, valid)
            X = X[m]; y = y[m]

            good = np.isfinite(X).all(1)
            X = X[good]; y = y[good]
            g = np.full(y.shape, gid, dtype=np.int32)  # group id = tile id
            Xs.append(X); ys.append(y); gs.append(g)

    X = np.vstack(Xs); y = np.concatenate(ys); groups = np.concatenate(gs)
    return X, y, groups

pairs = pair_images_labels(TRAIN_DIR)
print(f"[INFO] Found {len(pairs)} image/label pairs.")
X, y, groups = build_training_matrix_with_groups(pairs, keep_bands=KEEP_BANDS, valid=(0,1,2,3,4,5))
vals, cnts = np.unique(y, return_counts=True)
print("[INFO] Label distribution:", dict(zip(vals.tolist(), cnts.tolist())))
print("[INFO] X shape:", X.shape)

[INFO] Found 71 image/label pairs.
[INFO] Label distribution: {0: 272744, 1: 480, 2: 542, 3: 7010, 4: 8066, 5: 1968}
[INFO] X shape: (290810, 4)


Training + testing

In [ ]:
import numpy as np
from collections import defaultdict

# for each group (tile), record its dominant class
group_to_labels = defaultdict(list)
for gid in np.unique(groups):
    ys = y[groups == gid]
    vals, cnts = np.unique(ys, return_counts=True)
    dom = vals[np.argmax(cnts)]
    group_to_labels[gid] = (dom, len(ys))

# build a test set of groups that covers all classes
rng = np.random.default_rng(RANDOM_STATE)
all_groups = np.unique(groups)
classes = np.unique(y)

test_groups = []
remaining = set(all_groups)

# take at least one group per class
for c in classes:
    cand = [g for g in remaining if group_to_labels[g][0] == c]
    if cand:
        gpick = rng.choice(cand)
        test_groups.append(gpick)
        remaining.remove(gpick)

# top up test groups until ~TEST_SIZE of pixels
target = int(TEST_SIZE * X.shape[0])
pixels_in_test = sum(group_to_labels[g][1] for g in test_groups)
while pixels_in_test < target and remaining:
    gpick = rng.choice(list(remaining))
    test_groups.append(gpick)
    remaining.remove(gpick)
    pixels_in_test += group_to_labels[gpick][1]

test_mask = np.isin(groups, test_groups)
train_mask = ~test_mask

X_tr, X_te = X[train_mask], X[test_mask]
y_tr, y_te = y[train_mask], y[test_mask]
g_tr, g_te = groups[train_mask], groups[test_mask]

# sanity check: no overlap and classes exist in test
assert np.intersect1d(g_tr, g_te).size == 0
print("TRAIN classes:", np.unique(y_tr, return_counts=True))
print("TEST  classes:", np.unique(y_te, return_counts=True))

dict_weights = {0: 1, 1: 5, 2: 5, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1}

rf = RandomForestClassifier(
    n_estimators=N_ESTIMATORS,
    max_depth=MAX_DEPTH,
    n_jobs=-1,
    random_state=RANDOM_STATE,
    class_weight=dict_weights
)

rf.fit(X_tr, y_tr)

y_hat = rf.predict(X_te)
cm = confusion_matrix(y_te, y_hat, labels=[0,1,2,3,4,5])
report = classification_report(
    y_te, y_hat, labels=[1,2,3,4,5],
    target_names=["0","sparse(1)","dense(2)","3","4","5"],
    zero_division=0
)
kappa = cohen_kappa_score(y_te, y_hat)

print("Cohen's κ:", round(kappa, 4))
print("\nClassification report:\n", report)
print("Confusion matrix (rows=true, cols=pred):\n", cm)

MODEL_PATH = OUT_DIR / "rf_seagrass_model.joblib"
joblib.dump({"model": rf, "keep_bands": KEEP_BANDS, "class_map": {0:"other",1:"sparse",2:"dense",3:"x",4:"water",5:"y"}}, MODEL_PATH)
print(f"[OK] Saved model → {MODEL_PATH}")

plt.figure(figsize=(5,4))
im = plt.imshow(cm, interpolation="nearest")
plt.title("Confusion Matrix")
plt.xlabel("Predicted"); plt.ylabel("True")
plt.xticks([0,1,2,3,4,5], ["0","1","2","3","4","5"]); plt.yticks([0,1,2,3,4,5], ["0","1","2","3","4","5"])
for (i,j), v in np.ndenumerate(cm):
    plt.text(j, i, int(v), ha='center', va='center')
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.tight_layout()
cm_png = FIG_DIR / "confusion_matrix.png"
plt.savefig(cm_png, dpi=160); plt.close()
print(f"[OK] Saved {cm_png}")


TRAIN classes: (array([0, 1, 2, 3, 4, 5], dtype=uint16), array([215915,    335,    457,   4597,   6329,   1740]))
TEST  classes: (array([0, 1, 2, 3, 4, 5], dtype=uint16), array([56829,   145,    85,  2413,  1737,   228]))


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:2687: UserWarning: labels size, 5, does not match size of target_names, 6
  warnings.warn(


Cohen's κ: 0.9482

Classification report:
               precision    recall  f1-score   support

           0       0.86      1.00      0.92       145
   sparse(1)       0.79      1.00      0.88        85
    dense(2)       0.99      0.90      0.94      2413
           3       0.99      0.95      0.97      1737
           4       0.99      0.99      0.99       228

   micro avg       0.98      0.92      0.95      4608
   macro avg       0.92      0.97      0.94      4608
weighted avg       0.98      0.92      0.95      4608

Confusion matrix (rows=true, cols=pred):
 [[56738    24    23    24    18     2]
 [    0   145     0     0     0     0]
 [    0     0    85     0     0     0]
 [  253     0     0  2160     0     0]
 [   94     0     0     0  1643     0]
 [    2     0     0     0     0   226]]
[OK] Saved model → /content/seagrass_rf_output/rf_seagrass_model.joblib
[OK] Saved /content/seagrass_rf_output/figures/confusion_matrix.png


Writing out outputs

In [ ]:
WRITE_PROB_RASTER = True
TILE_SIZE = 1024

bundle = joblib.load(MODEL_PATH)
rf = bundle["model"]
KEEP_BANDS = bundle["keep_bands"]

def predict_raster(aoi_path: Path, model, use_bands=4, tile_size=1024, write_prob=False):
    with rasterio.open(aoi_path) as ds:
        B = min(use_bands, ds.count)
        H, W = ds.height, ds.width

        # outputs
        pred_full = np.zeros((H, W), dtype=np.uint8)
        prob_full = None
        if write_prob:
            prob_full = np.zeros((6, H, W), dtype=np.float32)

        for y0 in range(0, H, tile_size):
            h = min(tile_size, H - y0)
            for x0 in range(0, W, tile_size):
                w = min(tile_size, W - x0)
                win = Window(x0, y0, w, h)
                block = ds.read(list(range(1, B+1)), window=win)
                X = block.reshape(B, -1).T
                good = np.isfinite(X).all(1)
                pred = np.zeros(X.shape[0], dtype=np.uint8)
                if good.any():
                    if write_prob:
                        P = model.predict_proba(X[good])
                        Pfull = np.zeros((X.shape[0], 6), dtype=np.float32)
                        Pfull[good, :] = P
                        prob_tile = Pfull.reshape(h, w, 6).transpose(2,0,1)
                        prob_full[:, y0:y0+h, x0:x0+w] = prob_tile
                    pred_good = model.predict(X[good]).astype(np.uint8)
                    pred[good] = pred_good
                pred_full[y0:y0+h, x0:x0+w] = pred.reshape(h, w)

        # write class raster
        prof = ds.profile.copy()
        prof.update(count=1, dtype="uint8", nodata=0)
        out_path = PRED_DIR / f"{aoi_path.stem}_seagrass_pred.tif"
        with rasterio.open(out_path, "w", **prof) as dst:
            dst.write(pred_full, 1)

        prob_path = None
        if write_prob and prob_full is not None:
            pprof = ds.profile.copy()
            pprof.update(count=6, dtype="float32", nodata=np.nan)
            prob_path = PRED_DIR / f"{aoi_path.stem}_seagrass_prob.tif"
            with rasterio.open(prob_path, "w", **pprof) as dst:
                dst.write(prob_full)

        return out_path, prob_path

aoi_list = find_rasters(AOI_DIR)
print(f"[INFO] Found {len(aoi_list)} AOI rasters.")
predicted = []
for aoi in aoi_list:
    try:
        cls_path, prob_path = predict_raster(aoi, rf, use_bands=KEEP_BANDS, tile_size=TILE_SIZE, write_prob=WRITE_PROB_RASTER)
        print(f"[OK] Predicted → {cls_path.name}")
        if prob_path:
            print(f"    Probs  → {prob_path.name}")
        predicted.append((aoi, cls_path, prob_path))
    except Exception as e:
        print(f"[WARN] Prediction failed for {aoi.name}: {e}")


[INFO] Found 23 AOI rasters.
[OK] Predicted → 00_Landsat7_Composite_C2_1999_2000_summer_seagrass_pred.tif
    Probs  → 00_Landsat7_Composite_C2_1999_2000_summer_seagrass_prob.tif
[OK] Predicted → 01_Landsat7_Composite_C2_2000_2001_summer_seagrass_pred.tif
    Probs  → 01_Landsat7_Composite_C2_2000_2001_summer_seagrass_prob.tif
[OK] Predicted → 02_Landsat7_Composite_C2_2001_2002_summer_seagrass_pred.tif
    Probs  → 02_Landsat7_Composite_C2_2001_2002_summer_seagrass_prob.tif
[OK] Predicted → 03_Landsat7_Composite_C2_2002_2003_summer_seagrass_pred.tif
    Probs  → 03_Landsat7_Composite_C2_2002_2003_summer_seagrass_prob.tif
[OK] Predicted → 04_Landsat5_Composite_C2_2003_2004_summer_seagrass_pred.tif
    Probs  → 04_Landsat5_Composite_C2_2003_2004_summer_seagrass_prob.tif
[OK] Predicted → 05_Landsat5_Composite_C2_2004_2005_summer_seagrass_pred.tif
    Probs  → 05_Landsat5_Composite_C2_2004_2005_summer_seagrass_prob.tif
[OK] Predicted → 06_Landsat5_Composite_C2_2005_2006_summer_seagrass_pre

Making the map

In [ ]:
def match_rgb(aoi_path: Path) -> Path|None:
    m = re.match(r"^(\d+)_", aoi_path.name)
    if m:
        idx = m.group(1)
        cands = sorted(RGB_DIR.rglob(f"{idx}_*.tif"))
        if cands:
            return cands[0]
    cands = sorted(RGB_DIR.rglob(f"{aoi_path.stem}.tif"))
    if cands:
        return cands[0]
    return None

def rgb_preview(path: Path, max_size=1600):
    with rasterio.open(path) as ds:
        arr = ds.read()
        B,H,W = arr.shape
        if B >= 3:
            rgb = np.stack([arr[0],arr[1],arr[2]], axis=0).astype(np.float32)
        else:
            rgb = np.repeat(arr[:1], 3, axis=0).astype(np.float32)
        for i in range(3):
            band = rgb[i]
            finite = np.isfinite(band)
            if not finite.any():
                continue
            lo, hi = np.percentile(band[finite], (2,98))
            if hi > lo:
                band = np.clip((band - lo)/(hi-lo), 0, 1)
            else:
                band = np.zeros_like(band)
            rgb[i] = band
        rgb = np.transpose(rgb, (1,2,0))
        # simple resize to max dimension
        scale = min(1.0, max_size / max(H, W))
        if scale < 1.0:
            new_h = int(H*scale); new_w = int(W*scale)
            ys = (np.linspace(0, H-1, new_h)).astype(int)
            xs = (np.linspace(0, W-1, new_w)).astype(int)
            rgb = rgb[ys][:, xs]
    return rgb

def make_static_overlay(rgb_path: Path, pred_path: Path, out_png: Path):
    # load prediction raster (as full-res array)
    with rasterio.open(pred_path) as ds:
        pred = ds.read(1)  # (H,W)
    rgb = rgb_preview(rgb_path)
    # resize pred to preview
    new_h, new_w = rgb.shape[0], rgb.shape[1]
    ys = (np.linspace(0, pred.shape[0]-1, new_h)).astype(int)
    xs = (np.linspace(0, pred.shape[1]-1, new_w)).astype(int)
    pred_small = pred[ys][:, xs]

    # 0=transparent, 1=yellow, 2=green
    cmap = ListedColormap([(0,0,0,0), (1,1,0,0.6), (0,1,0,0.6)])
    norm = BoundaryNorm([-0.5,0.5,1.5,2.5], cmap.N)

    plt.figure(figsize=(9,9))
    plt.imshow(rgb)
    plt.imshow(pred_small, cmap=cmap, norm=norm)
    # NOTE: ADD KEY TO MAP FOR SEAGRASS COLOURS
    plt.title("Seagrass Prediction")
    plt.axis("off"); plt.tight_layout()
    plt.savefig(out_png, dpi=180); plt.close()

def make_folium_map(pred_path: Path, html_out: Path):
    # build RGB image
    with rasterio.open(pred_path) as ds:
        pred = ds.read(1)
        h,w = pred.shape
        rgba = np.zeros((h,w,4), dtype=np.uint8)
        rgba[pred==1] = np.array([255,255,0,140], dtype=np.uint8)
        rgba[pred==2] = np.array([0,255,0,140], dtype=np.uint8)
        overlay_png = html_out.with_suffix(".overlay.png")
        iio.imwrite(overlay_png, rgba)

        # compute WGS84 bounds from the raster bounds
        src_crs = ds.crs.to_string() if ds.crs else "EPSG:4326"
        (minx, miny, maxx, maxy) = ds.bounds
        (wgs_minx, wgs_miny, wgs_maxx, wgs_maxy) = transform_bounds(src_crs, "EPSG:4326", minx, miny, maxx, maxy, densify_pts=21)
        bounds = [[wgs_miny, wgs_minx], [wgs_maxy, wgs_maxx]]
        center = [(bounds[0][0]+bounds[1][0])/2, (bounds[0][1]+bounds[1][1])/2]

    m = folium.Map(location=center, zoom_start=13, control_scale=True)
    folium.raster_layers.ImageOverlay(
        name="Seagrass Prediction",
        image=str(overlay_png),
        bounds=bounds,
        opacity=0.7,
        interactive=True,
        cross_origin=False,
        zindex=2
    ).add_to(m)
    folium.LayerControl().add_to(m)
    m.save(html_out)
    return html_out

# generate overlays/maps for each AOI prediction
for aoi_path, cls_path, _prob in predicted:
    rgb_path = match_rgb(aoi_path) or aoi_path
    out_png = FIG_DIR / f"{aoi_path.stem}_overlay.png"
    try:
        make_static_overlay(rgb_path, cls_path, out_png)
        print(f"[OK] Static overlay → {out_png.name}")
    except Exception as e:
        print(f"[WARN] Static overlay failed for {aoi_path.name}: {e}")

    html_out = MAP_DIR / f"{aoi_path.stem}_map.html"
    try:
        make_folium_map(cls_path, html_out)
        print(f"[OK] Map → {html_out.name}")
    except Exception as e:
        print(f"[WARN] Map failed for {aoi_path.name}: {e}")


[OK] Static overlay → 00_Landsat7_Composite_C2_1999_2000_summer_overlay.png
[OK] Map → 00_Landsat7_Composite_C2_1999_2000_summer_map.html
[OK] Static overlay → 01_Landsat7_Composite_C2_2000_2001_summer_overlay.png
[OK] Map → 01_Landsat7_Composite_C2_2000_2001_summer_map.html
[OK] Static overlay → 02_Landsat7_Composite_C2_2001_2002_summer_overlay.png
[OK] Map → 02_Landsat7_Composite_C2_2001_2002_summer_map.html
[OK] Static overlay → 03_Landsat7_Composite_C2_2002_2003_summer_overlay.png
[OK] Map → 03_Landsat7_Composite_C2_2002_2003_summer_map.html
[OK] Static overlay → 04_Landsat5_Composite_C2_2003_2004_summer_overlay.png
[OK] Map → 04_Landsat5_Composite_C2_2003_2004_summer_map.html
[OK] Static overlay → 05_Landsat5_Composite_C2_2004_2005_summer_overlay.png
[OK] Map → 05_Landsat5_Composite_C2_2004_2005_summer_map.html
[OK] Static overlay → 06_Landsat5_Composite_C2_2005_2006_summer_overlay.png
[OK] Map → 06_Landsat5_Composite_C2_2005_2006_summer_map.html
[OK] Static overlay → 07_Landsat5_